# run_rakesh_portfolio_sim_updated.ipynb

This notebook (updated) fetches historical prices for a small set of real stocks using `yfinance`, builds a Binary Quadratic Model (BQM) for a simple portfolio selection (0/1 per stock), runs a classical simulated annealer (`neal` if available, otherwise a dimod fallback), and prints top feasible portfolios within a budget. It includes a Hindi explanatory section about which stocks the model tends to *fail* (reject) and why — useful for documentation and PR.

**How to use:**
- Open this notebook in JupyterLab / Jupyter Notebook.
- Activate your virtualenv: `source .venv/bin/activate`.
- Install dependencies (one-time): `pip install yfinance dimod dwave-neal numpy pandas` (or use dimod fallback).
- Run cells top-to-bottom.

> This notebook is simulation-only. Do **not** execute trades from results here. Tune penalty and budget before using for real decisions.

In [1]:
# Install dependencies (run this cell once in your environment)
# Uncomment to install (run only if needed)
# !pip install yfinance dimod dwave-neal numpy pandas
print('If needed, uncomment the pip install line and run this cell.')

If needed, uncomment the pip install line and run this cell.


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import dimod
import yfinance as yf
from datetime import datetime

# Sampler fallback logic: try neal, then dwave.samplers, then dimod.reference
try:
    import neal
    SamplerClass = neal.SimulatedAnnealingSampler
    sampler_source = 'neal.SimulatedAnnealingSampler'
except Exception:
    try:
        from dwave.samplers import SimulatedAnnealingSampler
        SamplerClass = SimulatedAnnealingSampler
        sampler_source = 'dwave.samplers.SimulatedAnnealingSampler'
    except Exception:
        from dimod.reference.samplers import SimulatedAnnealingSampler
        SamplerClass = SimulatedAnnealingSampler
        sampler_source = 'dimod.reference.samplers.SimulatedAnnealingSampler'

print(f"[{datetime.now().isoformat()}] Using sampler: {sampler_source}")

[2025-11-24T11:39:04.806069] Using sampler: neal.SimulatedAnnealingSampler


## Configuration
Edit the cell below to set Rakesh's tickers and parameters. You can also provide a `tickers.csv` file (one ticker per line) in the same folder to override the default list.

In [3]:
# ----------------------------
# CONFIG — change these values
# ----------------------------
# DEFAULT_TICKERS = ["ADANIPORTS.NS", "AXISBANK.NS", "BHARTIARTL.NS", "COALINDIA.NS", "TCS.NS", "RELIANCE.NS"]
DEFAULT_TICKERS = ["JUBLFOOD.NS","POLYPLEX.NS", "SULA.NS", "IGPL.NS", "AWL.NS", "BUTTERFLY.NS"]
TICKERS_FILE = "tickers.csv"  # optional override file

BUDGET = 50000        # ₹ example; set Rakesh's budget
RISK_FACTOR = 1.0     # tune 0.0..2.0 (higher => more risk-averse)
HISTORY_PERIOD = "1y" # e.g., "6mo", "1y", "2y"
NUM_READS = 2000      # simulated annealer reads (increase for better exploration)
PENALTY_MULT = 1e3

print('Config loaded. Edit variables in this cell as needed.')

Config loaded. Edit variables in this cell as needed.


In [4]:
def load_tickers(defaults, filename):
    if os.path.exists(filename):
        try:
            df = pd.read_csv(filename, header=None)
            ticks = [str(x).strip() for x in df[0].tolist() if str(x).strip()]
            if len(ticks) >= 1:
                print(f"Loaded {len(ticks)} tickers from {filename}")
                return ticks
        except Exception as e:
            print('Failed to read tickers.csv, falling back to defaults:', e)
    print(f"Using default tickers list ({len(defaults)}):", defaults)
    return defaults

# load tickers
tickers = load_tickers(DEFAULT_TICKERS, TICKERS_FILE)
print('Tickers to use:', tickers)

Using default tickers list (6): ['JUBLFOOD.NS', 'POLYPLEX.NS', 'SULA.NS', 'IGPL.NS', 'AWL.NS', 'BUTTERFLY.NS']
Tickers to use: ['JUBLFOOD.NS', 'POLYPLEX.NS', 'SULA.NS', 'IGPL.NS', 'AWL.NS', 'BUTTERFLY.NS']


## Fetch historical prices
This cell downloads adjusted close prices using `yfinance`. It explicitly requests `auto_adjust=False` and falls back to `'Close'` if `'Adj Close'` is unavailable (recent yfinance behavior change). The data is forward-filled and cleaned.

In [5]:
print('Fetching historical data from yfinance (period =', HISTORY_PERIOD, ') ...')

df_full = yf.download(
    tickers,
    period=HISTORY_PERIOD,
    interval='1d',
    progress=False,
    auto_adjust=False
)

# Ensure column exists
if 'Adj Close' in df_full.columns:
    df_price = df_full['Adj Close']
else:
    print("Warning: 'Adj Close' not found, falling back to 'Close'")
    df_price = df_full['Close']

# If single ticker, yf returns Series — convert to DataFrame
if isinstance(df_price, pd.Series):
    df_price = df_price.to_frame(tickers[0])

# forward-fill and drop rows with NaNs
df_price = df_price.ffill().dropna(axis=0, how='any')
print('Historical rows collected:', df_price.shape[0])

latest_price = df_price.iloc[-1].values.astype(float)
daily_ret = df_price.pct_change().dropna()
mu_daily = daily_ret.mean().values
cov_daily = daily_ret.cov().values
# annualize
mu = mu_daily * 252.0
cov = cov_daily * 252.0
print('Latest prices:', dict(zip(tickers, np.round(latest_price,2))))
print('Sample annualized expected returns (first 6):', np.round(mu[:min(6,len(mu))],4))

Fetching historical data from yfinance (period = 1y ) ...
Historical rows collected: 250
Latest prices: {'JUBLFOOD.NS': np.float64(273.8), 'POLYPLEX.NS': np.float64(704.8), 'SULA.NS': np.float64(377.35), 'IGPL.NS': np.float64(584.55), 'AWL.NS': np.float64(845.75), 'BUTTERFLY.NS': np.float64(237.65)}
Sample annualized expected returns (first 6): [-0.018  -0.0312 -0.3138 -0.0352 -0.283  -0.4577]


## Build BQM
Objective: minimize energy = -sum(mu_i * x_i) + risk_factor * x^T cov x
Constraint (soft): sum(price_i * x_i) <= budget with penalty P*(violation)^2
We convert the penalty expansion into linear and quadratic terms and build `dimod.BinaryQuadraticModel`.

In [6]:
# Build BQM
n = len(tickers)
linear = {i: -float(mu[i]) for i in range(n)}
quad = {}
for i in range(n):
    for j in range(i, n):
        val = float(RISK_FACTOR * cov[i, j])
        if i == j:
            linear[i] = linear.get(i, 0.0) + val
        else:
            quad[(i, j)] = quad.get((i, j), 0.0) + val

max_lin = max(abs(v) for v in linear.values()) if linear else 1.0
P = max_lin * 10.0 / (max(latest_price) * max(1, n))
P = P * PENALTY_MULT
print(f'Using penalty P = {P:.3g} (heuristic). Tune if infeasible solutions appear.')

for i in range(n):
    linear[i] = linear.get(i, 0.0) + P * (latest_price[i] ** 2) - P * 2 * BUDGET * latest_price[i]
for i in range(n):
    for j in range(i + 1, n):
        quad[(i, j)] = quad.get((i, j), 0.0) + P * 2 * latest_price[i] * latest_price[j]

bqm = dimod.BinaryQuadraticModel(linear, quad, 0.0, vartype=dimod.BINARY)
print('BQM built: variables =', n, ', interactions =', len(quad))

Using penalty P = 1.11 (heuristic). Tune if infeasible solutions appear.
BQM built: variables = 6 , interactions = 15


## Sample with simulator
This cell runs the sampler (`neal` if available, otherwise fallback). Increase `NUM_READS` for better exploration. The cell aggregates unique solutions and evaluates feasibility and metrics.

In [7]:
sampler = SamplerClass()
print('Sampling with num_reads =', NUM_READS, '...')
sampleset = sampler.sample(bqm, num_reads=NUM_READS)

agg = sampleset.aggregate()
rows = []
for rec in agg.record:
    sample_vals = rec[0]
    energy = float(rec[1])
    sample = {i: int(sample_vals[idx]) for idx, i in enumerate(agg.variables)}
    x = np.array([sample[i] for i in range(n)])
    total_price = float(np.dot(latest_price, x))
    exp_return = float(np.dot(mu, x))
    risk = float(x @ cov @ x)
    rows.append({
        'picks': [tickers[i] for i in range(n) if x[i] == 1],
        'x': x,
        'price': total_price,
        'exp_return': exp_return,
        'risk': risk,
        'energy': energy
    })

import pandas as pd

df = pd.DataFrame(rows)
df = df.drop_duplicates(subset=['price','exp_return','risk','energy']).reset_index(drop=True)
df['feasible'] = df['price'] <= BUDGET
df = df.sort_values(['feasible','energy'], ascending=[False, True]).reset_index(drop=True)

print('\nTop feasible solutions (within budget):')
feasible = df[df['feasible']]
if feasible.empty:
    print('No feasible solution found. Try increasing penalty P or adjusting budget/inputs.')
else:
    for idx, r in feasible.head(10).iterrows():
        print(f"{idx+1}: picks={r['picks']}, price={r['price']:.2f}, exp_return={r['exp_return']:.4f}, risk={r['risk']:.4f}, energy={r['energy']:.4f}")

print('\nTop overall by energy (may include infeasible):')
for i, r in df.head(5).iterrows():
    print(f"{i+1}: picks={r['picks']}, price={r['price']:.2f}, feasible={r['feasible']}, energy={r['energy']:.4f}")

Sampling with num_reads = 2000 ...

Top feasible solutions (within budget):
1: picks=['JUBLFOOD.NS', 'POLYPLEX.NS', 'SULA.NS', 'IGPL.NS', 'AWL.NS', 'BUTTERFLY.NS'], price=3023.90, exp_return=-1.1389, risk=1.6472, energy=-324395341.5088

Top overall by energy (may include infeasible):
1: picks=['JUBLFOOD.NS', 'POLYPLEX.NS', 'SULA.NS', 'IGPL.NS', 'AWL.NS', 'BUTTERFLY.NS'], price=3023.90, feasible=True, energy=-324395341.5088


## 📌 मॉडल में कौन से स्टॉक्स FAIL होते हैं और क्यों? (BQM के आधार पर)

**(Pure Mathematical / BQM-based Explanation — No Trading Advice)**

यह मॉडल किसी भी स्टॉक को खरीदने या न खरीदने का निर्णय **pure mathematics** के आधार पर करता है।  
यह *financial advice नहीं* है, बल्कि सिर्फ BQM (Binary Quadratic Model) द्वारा चुने गए ऊर्जा स्तर (energy), risk-return ratio और covariance के आधार पर filtering है।

### 1️⃣ Negative Expected Return वाले स्टॉक्स  
यदि किसी स्टॉक के पिछले 6–12 महीनों का average return negative है,  
तो उसका linear term (–mu) model में positive बन जाता है,  
और energy बढ़ती है → मॉडल उसे pick नहीं करता।

**इसका मतलब:** स्टॉक पिछले समय में गिरा हुआ हो → model reject करता है।

---

### 2️⃣ बहुत High Risk / High Variance वाले स्टॉक्स  
Covariance matrix में जिन स्टॉक्स का variance बहुत ज्यादा होता है,  
उनका risk term xᵀ Σ x बहुत बड़ा हो जाता है,  
जो energy को बढ़ाता है → model reject करता है।

**इसका मतलब:** जिनमें तेज़ ऊपर-नीचे चलन हो → model उन्हें avoid करता है।

---

### 3️⃣ ऐसे स्टॉक्स जिनकी volatility दूसरों से बहुत ज़्यादा correlated हो  
अगर एक स्टॉक कई और स्टॉक्स के साथ बहुत high covariance रखता है,  
तो उसे जोड़ते ही पूरा portfolio का risk बढ़ जाता है।

**इसका मतलब:** जो stock “group में risk फैलाता है” → fail हो जाता है।

---

### 4️⃣ Price बहुत बड़ा हो, लेकिन return कम हो  
यदि किसी stock का price बहुत ऊँचा है, और उसका expected return (mu) बहुत कम है,  
तो **(return / price)** ratio खराब हो जाता है और budget-constraint के साथ energy बहुत बढ़ जाती है → fail.

**इसका मतलब:** महँगे लेकिन कम-return वाले स्टॉक्स automatically reject.

---

### 5️⃣ Penny-type (बहुत low price) वाले noisy स्टॉक्स  
कई बार बहुत low price वाले स्टॉक्स (₹20–₹30 से नीचे) में return भले बड़ा दिखे, लेकिन covariance (risk) high होता है।  
High risk term → fail.

**इसका मतलब:** शोर/उछाल वाले स्टॉक्स बजट में fit होने के बावजूद reject हो जाते हैं।

---

### 6️⃣ Budget constraint के कारण चुनने पर नुकसान होता है  
अगर किसी स्टॉक को चुनने से P*(price_sum – budget)² penalty बहुत बढ़ जाता है,  
तो model automatically उसे छोड़ देता है।

**इसका मतलब:** Budget brake लगते ही कुछ stocks अपने आप बाहर हो जाते हैं।

---

## ⭐ Model Fail का आसान Summary  
मॉडल ऐसे stocks को FAIL करता है:

| FAIL कारण | Meaning |
|----------|----------|
| ❌ Negative mu | पिछले returns खराब |
| ❌ High variance | बहुत जोखिम / volatility |
| ❌ High covariance | दूसरों को भी risky बना देता है |
| ❌ Bad return/price ratio | महँगा लेकिन कम फायदा |
| ❌ Penny noise | बहुत low price, high noise |
| ❌ Budget penalty | बजट से बाहर जाने पर energy बढ़ जाती है |

---

**Important (Safe Explanation):** यह *trading recommendation नहीं है* — मॉडल सिर्फ गणित के आधार पर बताता है कि **कौन-सा स्टॉक इस optimization में useful है** और कौन-सा नहीं।

---

If you want, I can also run a quick classification on a list of tickers you provide and mark which ones are likely to be 'fail' for this model (purely mathematical).

## How to switch to D-Wave QPU later
When you receive your Leap API token and configure auth (`dwave auth login` or env vars), replace the sampler creation cell with the following:

```python
from dwave.system import DWaveSampler, EmbeddingComposite
sampler = EmbeddingComposite(DWaveSampler())
# then call: sampleset = sampler.sample(bqm, num_reads=NUM_READS)
```

No other changes are required to the BQM or parsing logic.